# MusicAI
## Sztuczna Inteligencja - projekt 
### Część 2 - przygotowanie danych
Autorzy: Jakub Ochnik, Adam Karabiniewicz, Marcel Bieniek
___


Importing necessary libraries and packages

In [ ]:
import librosa
import math
import os
import json

Defining constants

In [ ]:
DATASET_PATH = "Data\\genres_original\\"
JSON_PATH = "Data\\data.json"

SAMPLE_RATE = 22050
DURATION = 30 # seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

SET_SIZE = 0.25

A function which generates a JSON dataset

In [ ]:
def export_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
    # main data structure
    data = {
        "mapping": [], # different genre labels
        "mfcc": [], # training inputs
        "labels": [] # outputs, targets
    }

    num_samples_per_segment = SAMPLES_PER_TRACK // num_segments
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length)

    # loop through all the genres
    for i, (dirpath, dirname, filenames) in enumerate(os.walk(dataset_path)):
        if dirpath != dataset_path:
            # save the semantic label - mapping
            dirpath_components = dirpath.split("\\") # e.g genre/blues => ["genre, "blues"]
            semantic_label = dirpath_components[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing {}".format(semantic_label))

            # process files for each specific genre
            file_num = 0
            for f in filenames:
                if file_num > math.ceil(len(filenames) * SET_SIZE):
                    break
                file_num += 1
                # load audio file
                file_path = os.path.join(dirpath, f)
                signal, sr = librosa.load(file_path, sr = SAMPLE_RATE)               
                
                # divide into segments
                for s in range(num_segments):
                    start_sample = num_samples_per_segment * s
                    finish_sample = start_sample + num_samples_per_segment

                    mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample], sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
                    mfcc = mfcc.T

                    # check expected length
                    if len(mfcc) == expected_num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist()) #np array -> list
                        data["labels"].append(i-1)
                        print("{}, segment: {}".format(file_path, s))
    
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)

Generate the dataset (careful, almost 20 million lines, 600+ MB)

In [ ]:
export_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)